### Import the necessary database

In [ ]:
import numpy as np
import xarray as xr
import pandas as pd
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
import cartopy.feature as cfeature

In [ ]:
#In[2]:
# define function
import src.SAT_function as data_process
import src.Data_Preprocess as preprosess

In [ ]:
# import src.slurm_cluster as scluster
# client, scluster = scluster.init_dask_slurm_cluster(scale=4, cores=50, memory="200GB")

In [ ]:
# input the data
dir_in = '/work/mh0033/m301036/Land_surf_temp/Disentangling_OBS_SAT_trend/Figure4/data/NAT_wrt_nh_Keil/'

In [ ]:
# Input the MMEM of SAT-OBS internal variability
ds = xr.open_mfdataset(dir_in + 'Internal_subpolar_gyre_wrt_NH_mean_trend_segmented.nc')

In [ ]:
ds

### specify the 5 and 95 percentile values for each year step and output the 10-73yr unforced percentile timeseires for each regions

In [ ]:
# define the function to calculate the percentile
def calc_percentile(da, q):
    """ Calculate the qth percentile of the data along the specified dimension.
    Args:
    da: xr.DataArray
    dim: str
    q: float
    Returns:
    xr.DataArray
    """
    # remove nans for da
    da = da.dropna(dim='segment')
    lower_percentile = np.percentile(da, q)
    upper_percentile = np.percentile(da, 100-q)
    
    return lower_percentile, upper_percentile

In [ ]:
ds['ICV_trend_11yr']

In [ ]:
# calculate the regional mean's percentile
vars_yr = np.arange(10,74)
print(vars_yr)
# 5%---[0]
unforced_trend_subpolar_gyre_lower_percentile = {}

# 95%---[1]
unforced_trend_subpolar_gyre_upper_percentile = {}

for var_yr in vars_yr:
    unforced_trend_subpolar_gyre_lower_percentile[str(var_yr)], unforced_trend_subpolar_gyre_upper_percentile[str(var_yr)] = calc_percentile(ds['ICV_trend_'+str(var_yr)+'yr'], 5)
    

In [ ]:
unforced_trend_subpolar_gyre_lower_percentile

In [ ]:
# check the seires for different time scales
unforced_trend_subpolar_gyre_lower_percentile['73'], unforced_trend_subpolar_gyre_upper_percentile['73']

In [ ]:
list(unforced_trend_subpolar_gyre_lower_percentile.values())

In [ ]:
# transform the dictionary to the dataarray
unforced_trend_subpolar_gyre_lower_percentile_da = xr.DataArray(list(unforced_trend_subpolar_gyre_lower_percentile.values()), coords=[('time', vars_yr)])
unforced_trend_subpolar_gyre_upper_percentile_da = xr.DataArray(list(unforced_trend_subpolar_gyre_upper_percentile.values()), coords=[('time', vars_yr)])

In [ ]:
# save the percentile data
dir_out = '/work/mh0033/m301036/Land_surf_temp/Disentangling_OBS_SAT_trend/Figure4/data/NAT_wrt_nh_Keil/'
unforced_trend_subpolar_gyre_lower_percentile_da.to_netcdf(dir_out+'internal_subpolar_gyre_trend_lower_percentile.nc')
unforced_trend_subpolar_gyre_upper_percentile_da.to_netcdf(dir_out+'internal_subpolar_gyre_trend_upper_percentile.nc')